## Cobot Projectile Obstacle Avoidance
Work Items:
1. Obstacle generator
2. YOLO image segmentation
3. Cobot control and DQN

In [1]:
import pybullet as p
import gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
import cobot_ai4robotics
import math

In [2]:
# Hyperparameters.
EPISODES = 2000 # Keep low at a modest number to start with. Can load and save to do more episodes.
LEARNING_RATE = 0.0025#0.00025 
MEM_SIZE = 50000
REPLAY_START_SIZE = 10000
BATCH_SIZE = 512#32 # Try upping the batch size for more complex space.
GAMMA = 0.99 # Discount factor.
EPS_START = 0.1
EPS_END = 0.0001
EPS_DECAY = 4 * MEM_SIZE
MEM_RETAIN = 0.1
NETWORK_UPDATE_ITERS = 500

FC1_DIMS = 128
FC2_DIMS = 128

# Training status metrics.
best_reward = 0
average_reward = 0
episode_history = []
episode_reward_history = []
max_ep_reward = []
np.bool = np.bool_

# Classes using deep Q-Learning to teach the robot to drive to the goal.
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim

#____________________________________________________DQN_STRUCTURE_____________________________________________________________________________#

# Class to define network structure for the target and policy networks. They MUST be of the same structure.
class Network(torch.nn.Module):
    # Initialise the network's structure.
    def __init__(self, env):
        super().__init__()
        self.input_shape = env.observation_space.shape[0]
        self.action_space = env.action_space.n

        # Network of hidden layers, number and structure tbd.
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(self.input_shape, FC1_DIMS),
            torch.nn.ReLU(),
            torch.nn.Linear(FC1_DIMS, FC2_DIMS),
            torch.nn.ReLU(),
            torch.nn.Linear(FC2_DIMS, self.action_space)
        )

        self.optimizer = optim.Adam(self.parameters(), lr=LEARNING_RATE) # No need to pick out a better optimiser, assume that this will do the job.
        self.loss = nn.MSELoss() # Using mean squared error as the loss. Should be alright without considering a different loss function.
    
    # Pass the states into the network to return a weighted list of action responses.
    def forward(self, x):
        return self.layers(x)
    
    #____________________________________________________REPLAY_BUFFER_____________________________________________________________________________#

# The replay buffer is used to store a dataset of the agent's experiences so it can be randomly sampled in batches to learn from.
class ReplayBuffer:
    # Initialise memory space for states, actions, rewards, next_states, and done flags.
    def __init__(self, env):
        self.mem_count = 0
        self.states = np.zeros((MEM_SIZE, env.observation_space.shape[0]), dtype=np.float32)
        self.actions = np.zeros(MEM_SIZE, dtype=np.int64)
        self.rewards = np.zeros(MEM_SIZE, dtype=np.float32)
        self.states_ = np.zeros((MEM_SIZE, env.observation_space.shape[0]), dtype=np.float32)
        self.dones = np.zeros(MEM_SIZE, dtype=np.bool)
    
    # Add an experience to the Replay Buffer.
    def add(self, state, action, reward, state_, done):
        if self.mem_count < MEM_SIZE:
            mem_index = self.mem_count
        else: # Attempt to avoid catastrophic forgetting by retaining a portion of the dataset when the replay buffer is full and needs to be cleared.
            mem_index = int(MEM_SIZE*MEM_RETAIN)
            self.mem_count = mem_index
            print("Overwriting Replay memory from", mem_index)

        # Add experience to specified mem_index entry.
        #print(state[0], self.states.shape)
        self.states[mem_index] = state
        self.actions[mem_index] = action
        self.rewards[mem_index] = reward
        self.states_[mem_index] = state_
        self.dones[mem_index] = 1 - done 

        self.mem_count += 1 # Increment index by 1 for next experience.

        # if (self.mem_count % 100) == 0:
        #     print("Memory Count: ", self.mem_count)

    # Return a random sample from the replay buffer of size BATCH_SIZE.
    def sample(self):
        MEM_MAX = min(self.mem_count, MEM_SIZE)
        # Get indices of size BATCH_SIZE from the replay buffer of current size MEM_MAX to train on.
        batch_indices = np.random.choice(MEM_MAX, BATCH_SIZE, replace=True) # Replace - you can select the same value more than once (returned to sample pool after being chosen).

        states = self.states[batch_indices]
        actions = self.actions[batch_indices]
        rewards = self.rewards[batch_indices]
        states_ = self.states_[batch_indices]
        dones = self.dones[batch_indices]

        return states, actions, rewards, states_, dones
    
#____________________________________________________DQN_SOLVER_____________________________________________________________________________#
class DQN_Solver:
    def __init__(self, env, load=None):
        self.memory = ReplayBuffer(env) # Access the replay buffer through the self.memory variable.
        self.policy_network = Network(env) # The Q or policy network.
        if load != None: # Load given model.
            self.policy_network.load_state_dict(torch.load(load))
        self.target_network = Network(env) # The Q^ or target network (its hat fell off a bit...)
        self.target_network.load_state_dict(self.policy_network.state_dict()) # When we initialise, have the target network copy the policy network's weights.
        self.learn_count = 0 # Iterations we have learnt for.
    
    # Our epsilon greedy function responsible for choosing actions as we learn. 
    def choose_action(self, observation):
        if self.memory.mem_count > REPLAY_START_SIZE:
            eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * self.learn_count / EPS_DECAY)
        else:
            eps_threshold = 1.0

        # If we roll a value lower than eps_threshold, take a random action in the action_space.
        # As part of part 2, we must use prior knowledge to set a non-uniform probability distribution for which random actions we take.
        if random.random() < eps_threshold:             #
            return np.random.choice(np.array(range(7)), p=[1/7,  1/7,  1/7,  1/7,  1/7,
                                                            1/7,  1/7])# p=[1/45, 1/45, 1/45, 1/45, 1/45, 1/45, 1/45, 27/45, 1/45]) # there are 9 actions (0-8). p = [probabilities * 9]
        
        # If we pass the random check and are past the initial filling of the replay buffer, begin to take intelligent action.
        state = torch.tensor(observation).float().detach()
        state = state.unsqueeze(0)
        self.policy_network.eval() # Sets the network into evaluation mode.
        with torch.no_grad(): # Disable gradient computation for the forward pass.
            q_values = self.policy_network.forward(state) # Given the state, get Q values for each action.
        return torch.argmax(q_values).item() # Return the action with the highest Q value.
    
    # Training Loop
    def learn(self):
        states, actions, rewards, states_, dones = self.memory.sample() # Random sample of BATCH_SIZE from replay buffer.
        states = torch.tensor(states , dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.long)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        states_ = torch.tensor(states_, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.bool)
        batch_indices = np.arange(BATCH_SIZE, dtype=np.int64)

        self.policy_network.train(True) # Sets the network into training mode.
        q_values = self.policy_network(states)  # First, q_values for all sampled experiences and all actions.
        q_values = q_values[batch_indices, actions] # Then, q_values for all sampled experiences and select actions. Check this?

        self.target_network.eval() # Evaluation mode for target network.
        with torch.no_grad():
            q_values_next = self.target_network.forward(states_)

        q_values_next_max = torch.max(q_values_next, dim=1)[0] # The maximum q values for the next state.

        q_target = rewards + GAMMA * q_values_next_max * dones # Target Q value.

        # Calculate loss on new q_values (policy network) and q_target (target_network).
        loss = self.policy_network.loss(q_values, q_target)

        # SGD.
        self.policy_network.optimizer.zero_grad() # Zero the gradient to prevent accumulation.
        loss.backward()
        self.policy_network.optimizer.step()
        self.learn_count += 1

        # Target network's weights should be set to policy network's weights every so often to keep learning with some stability.
        if self.learn_count % NETWORK_UPDATE_ITERS == NETWORK_UPDATE_ITERS -1:
            print("Updating Target Network")
            self.update_target_network()

    # Set Q^ = Q every so often.
    def update_target_network(self): 
        self.target_network.load_state_dict(self.policy_network.state_dict())

    def returning_epsilon(self): # Is this even needed?
        return self.exploration_rate

## Training Section

In [3]:
# Training DQN.
env = gym.make("cobot_ai4robotics", apply_api_compatibility=True, renders=True, isDiscrete=True, debug=0)

# Seed setting. Choose any number that makes you happy :)
# env.action_space.seed()
# random.seed()
# np.random.seed()
# torch.manual_seed()
episode_batch_score = 0
episode_reward = 0

# Comment and uncomment to pick whether to start a new model or train an existing one.
agent = DQN_Solver(env)
# agent = DQN_Solver(env, load="project_policy_network.pkl")
plt.clf() # Clear current figure.
goal_count = 0

# Training loop.
print("No. of eps: ", EPISODES)
for i in range(EPISODES):
    print("Episode", i)
    state, _ = env.reset()
    while True: #for j in range(200): # Set finite life for each EPISODE?
        # Loopa
        action = agent.choose_action(state)
        state_, reward, done, _, info = env.step(action)

        # New reward scheme per part 3.
        # Reward for reaching goal has been added to env.step in the gym environment.
        # if done: # Reached goal.
        #     reward = 50
        agent.memory.add(state, action, reward, state_, done)

        if agent.memory.mem_count > REPLAY_START_SIZE:
            agent.learn()
        
        state = state_ # Move current state to new state positions.
        episode_batch_score += reward
        episode_reward += reward

        if done & (reward > 0): # Count the number of episodes for which we reach the goal.
            goal_count += 1
            reward = 0 # Represent reward as distance in final graph.

        if done: # If we reached the goal, we can leave early.
            max_ep_reward.append(reward)
            break

    episode_history.append(i)   # Append episode number for plotting.
    episode_reward_history.append(episode_reward) # Append overall episode reward for plotting.

    print(agent.memory.mem_count, episode_reward)

    episode_reward = 0.0    # Reset episode reward for next round.

    # Save model progress every n batches.
    if i % 100 == 0 and agent.memory.mem_count > REPLAY_START_SIZE:
        torch.save(agent.policy_network.state_dict(), "project_policy_network.pkl")
        print("average total reward per episode batch since episode ", i, ": ", episode_batch_score/ float(100))
        episode_batch_score = 0
    elif agent.memory.mem_count < REPLAY_START_SIZE:
        #print("waiting for buffer to fill...")
        episode_batch_score = 0
print("Successful Episodes:", goal_count, "/", EPISODES)
plt.plot(episode_history, episode_reward_history)
plt.show()    

env.close()

YOLO LOADED
No. of eps:  2000
Episode 0


c:\Users\mvsue\miniconda3\envs\Ai4Robotics_FinalProject\lib\site-packages\gym\utils\passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\mvsue\miniconda3\envs\Ai4Robotics_FinalProject\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
c:\Users\mvsue\miniconda3\envs\Ai4Robotics_FinalProject\lib\site-packages\gym\utils\passive_env_checker.py:137: UserWarning: WARN: The obs returned by the `step()` method was expecting a numpy array, actual type: <class 'list'>
  logger.warn(
c:\Users\mvsue\miniconda3\envs\Ai4Robotics_FinalProject\lib\site-packages\gym\spaces\box.py:227: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")


Contact on table at point (0.75 -0.11 0.62) on KUKA.
46 -500
Episode 1
Contact on table at point (0.75 0.02 0.62) on KUKA.
94 -500.0
Episode 2
118 50.0
Episode 3
Contact by ball no. 4 at point (0.00 -0.12 1.41) on KUKA.
304 -518.5757602724461
Episode 4
Contact by ball no. 1 at point (0.32 -0.09 1.41) on KUKA.
368 -531.7106186898814
Episode 5
Contact by ball no. 0 at point (0.60 0.11 1.50) on KUKA.
390 -501.9227441616018
Episode 6
Contact on table at point (0.75 -0.07 0.62) on KUKA.
431 -500.0
Episode 7
Contact by ball no. 0 at point (0.43 -0.03 1.49) on KUKA.
454 -506.11590373230496
Episode 8
Contact by ball no. 1 at point (0.67 -0.20 1.40) on KUKA.
516 -513.328776551563
Episode 9
Contact on table at point (0.75 0.07 0.62) on KUKA.
558 -500.0
Episode 10
Contact on table at point (0.75 0.24 0.62) on KUKA.
590 -500.0
Episode 11
Contact on table at point (0.67 0.22 0.62) on KUKA.
617 -522.4076607116816
Episode 12
Contact on table at point (0.75 -0.21 0.62) on KUKA.
654 -500.0
Episode 13
C

: 

In [ ]:
print("Successful Episodes:", goal_count, "/", EPISODES)
plt.plot(episode_history, episode_reward_history)
plt.show()

## Demo Section

In [4]:
# Basic code to open the GUI environment window.
# https://www.gymlibrary.dev/content/environment_creation/ - make a custom gym environment.
# env.reset()
env = gym.make("cobot_ai4robotics", apply_api_compatibility=True, renders=True, isDiscrete=True, debug=0)
agent = DQN_Solver(env)
agent.policy_network.load_state_dict(torch.load("project_policy_network.pkl"))
# agent.policy_network.eval()

ob, _ = env.reset()

# Modify joint control so that it is a change to the current joint position, not specifying a joint position.
# origin = np.array([0,0,0,0,0,0,0,0,0])
while True:
    with torch.no_grad():
        q_values = agent.policy_network(torch.tensor(ob, dtype=torch.float32))
    action = torch.argmax(q_values).item()
    ob, reward, done, _, info = env.step(action)
    #ob, reward, done, _, info = env.step(env.action_space.sample()) # Things will not move unless you call env.step.
    print(ob)
    print(reward)
    if done:
        if reward > 0:
            print("Success!")
        break
    # First 7 items in ob are the pose. The next 4 numbers are the bounding box coordinates compressed to be from 0-1, and the last item is the distance to camera.
env.close()

YOLO LOADED
[3.132653890816124, 0.5133579396161561]
0.48664206038384394
[3.077027736650157, 0.5198091874999704]
0.4801908125000296
[3.0192943729554518, 0.5265540625757381]
0.4734459374242619
[2.964440077315589, 0.5335492700091201]
0.4664507299908799
[2.907507856554382, 0.5407958795461869]
0.4592041204538131
[2.8534748153566576, 0.5482845470817492]
0.45171545291825077
[2.820430028624644, 0.5402437453769664]
0.4597562546230336
[2.776040081215613, 0.54641598440301]
0.45358401559699
[2.7429778243895777, 0.5312888614083632]
0.4687111385916368
[2.712848516969404, 0.5175750039742643]
0.48242499602573574
[2.67980726578215, 0.5237680504502391]
0.4762319495497609
[2.646796911573935, 0.5394365651504763]
0.46056343484952367
[2.6262229982237084, 0.5346283134795043]
0.46537168652049565
[2.613282896318616, 0.5309524981972044]
0.46904750180279564
[2.588354992192583, 0.5475169772844802]
0.45248302271551977
[2.5800936787330433, 0.5517180379731678]
0.44828196202683224
[2.57017887222303, 0.550965628833999

In [ ]:
env.close()

error: Not connected to physics server.

Last resort:
Use getLinkStates to get the position of each link? The observation should be the closest distance from an obstacle to any link.
Add small reward for approaching goal.